In [1]:
import smoldyn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
def generate_params():
    abs_grad = np.random.uniform(0,1)
    kcatkm = np.random.exponential(3.5e-5,1)
    Paseconc = np.random.uniform(0,3000)
    return abs_grad, kcatkm[0], Paseconc

ion time step
 No errors
 41 total warnings



In [4]:
def set_Pase_rates(abs_grad, kcatkm, Paseconc):
    smol_Pase = Paseconc * 0.6022
    smol_kcatkm = kcatkm * 10 ** 9 * 1.7e-9
    mid_rate = smol_Pase * smol_kcatkm
    max_rate = mid_rate * 2 /(2-abs_grad)
    min_rate = max_rate * (1-abs_grad)
    rate_range=max_rate-min_rate
    for i in range(8):
        s.setReactionRate(f"pase_{i+1}",min_rate+rate_range*(7-i)/7, 0)


In [5]:
def add_totals(df):
    df['MEG3_total'] = df['MEG3'] + df['MEG3p'] + df['MEG3-mRNA']
    df['MEG3_fast'] = df['MEG3'] + df['MEG3p']
    return df

In [6]:
def scale_concs(anterior, posterior):
    colnames = ['MEG3', 'MEG3p', 'MEG3-mRNA', 'MEG3_fast', 'MEG3_total']
    for col in colnames:
         anterior[f'scaled_{col}'] = anterior[col]#/0.625
         posterior[f'scaled_{col}'] = posterior[col]#/0.375
    return anterior, posterior

In [7]:
def get_grads(anterior, posterior):
    grads_to_calc = ['scaled_MEG3', 'scaled_MEG3p', 'scaled_MEG3-mRNA', 'scaled_MEG3_fast', 'scaled_MEG3_total']
    grad_list=[]
    for species in grads_to_calc:
        grad = 1-(anterior[species].mean()/posterior[species].mean())
        grad_list.append(grad)
    return grad_list


In [98]:
def runSims(smoldyn_simulation, num_sims):
    data = pd.DataFrame({"abs_phosphatase_grad":[],"kcat/Km":[],"phosphatase_conc":[],"meg3_grad":[], \
                         "meg3p_grad":[],"meg3-rna_grad":[],"fast_meg3_grad":[],"total_meg3_grad":[]})
    for i in range(num_sims):
        abs_grad, kcatkm, Paseconc = (1, 1.8e-4,1225)#generate_params()
        set_Pase_rates(abs_grad, kcatkm, Paseconc)
        smoldyn_simulation.run(10000,10,overwrite=True)
        anterior = pd.read_csv('anterior.csv')
        posterior = pd.read_csv('posterior.csv')
        anterior = add_totals(anterior)
        posterior = add_totals(posterior)
        anterior, posterior = scale_concs(anterior, posterior)
        grad_list = get_grads(anterior, posterior)
        sim_data = pd.DataFrame({"abs_phosphatase_grad":[abs_grad],"kcat/Km":[kcatkm],"phosphatase_conc":[Paseconc], \
                                 "meg3_grad":[grad_list[0]],"meg3p_grad":[grad_list[1]], \
                                 "meg3-rna_grad":[grad_list[2]],"fast_meg3_grad":[grad_list[3]],\
                                 "total_meg3_grad":[grad_list[4]]})
        data = pd.concat([data, sim_data], ignore_index=True)
    return data




In [99]:
s = smoldyn.Simulation.fromFile('elegans_with_competition.txt')

--------------------------------------------------------------
Running Smoldyn 2.72.dev202401300921

CONFIGURATION FILE
 Path: '/home/participant/Course_Materials/Mini-Project/WithComp/'
 Name: 'elegans_with_competition.txt'
 Reading file: '/home/participant/Course_Materials/Mini-Project/WithComp/elegans_with_competition.txt'
 Reading file: '/home/participant/Course_Materials/Mini-Project/WithComp/mex5_grad.txt'
 Loaded file successfully
 setting up molecules
 setting up virtual boxes
 setting up compartments
 setting up reactions
 setting up surfaces

SIMULATION PARAMETERS
 file: /home/participant/Course_Materials/Mini-Project/WithComp/elegans_with_competition.txt
 starting clock time: Wed Mar  6 17:06:23 2024
 3 dimensions
 Random number seed: 1709744783
 Time from 0 to 10000 step 10

GRAPHICS PARAMETERS
 No graphical output

WALL PARAMETERS
 wall 0: dimension x, at -15
 wall 1: dimension x, at 40
 wall 2: dimension y, at -15
 wall 3: dimension y, at 15
 wall 4: dimension z, at -15
 

In [100]:
runSims(s, 1)

SIMULATION PARAMETERS
 file: /home/participant/Course_Materials/Mini-Project/WithComp/elegans_with_competition.txt
 starting clock time: Wed Mar  6 17:06:23 2024
 3 dimensions
 Random number seed: 1709744783
 Time from 0 to 10000 step 10

GRAPHICS PARAMETERS
 No graphical output

WALL PARAMETERS
 wall 0: dimension x, at -15
 wall 1: dimension x, at 40
 wall 2: dimension y, at -15
 wall 3: dimension y, at 15
 wall 4: dimension z, at -15
 wall 5: dimension z, at 15
 system volume: 49500
 system corners: (-15,-15,-15) and (40,15,15)

MOLECULE PARAMETERS
 2 molecule lists:
  diffuselist, fixedlist
 8 species defined:
 PGL3:
  solution: difc=5, rms step=10, list=diffuselist, number=0
 PGL3-mRNA:
  solution: difc=0.11, rms step=1.48324, list=diffuselist, number=0
 MEG3:
  solution: difc=5, rms step=10, list=diffuselist, number=1000
 MEG3-mRNA:
  solution: difc=0.11, rms step=1.48324, list=diffuselist, number=0
 MEG3p:
  solution: difc=5, rms step=10, list=diffuselist, number=0
 mRNA:
  solut

Libsmoldyn 'Smoldyn bug' error in smolSetReactionRate: RxnSetValue error
Libsmoldyn 'Smoldyn bug' error in smolSetReactionRate: RxnSetValue error
Libsmoldyn 'Smoldyn bug' error in smolSetReactionRate: RxnSetValue error
Libsmoldyn notification from smolRunSim: Simulation complete


,abs_phosphatase_grad,kcat/Km,phosphatase_conc,meg3_grad,meg3p_grad,meg3-rna_grad,fast_meg3_grad,total_meg3_grad
0,1.5,0.00018,1225.0,0.430976,-1.153473,0.476263,-0.018381,0.113363
